In [53]:
import math
import numpy as np
import torch
import gpytorch
import tqdm
import random
import time
from matplotlib import pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
import sys
sys.path.append("../")
sys.path.append("../directionalvi/utils")
sys.path.append("../directionalvi")
import traditional_vi
from RBFKernelDirectionalGrad import RBFKernelDirectionalGrad
#from DirectionalGradVariationalStrategy import DirectionalGradVariationalStrategy
from dfree_directional_vi import train_gp, eval_gp
from metrics import MSE
import testfun
from csv_dataset import csv_dataset

In [54]:
dataset = csv_dataset("../experiments/real_data/WECs_DataSet/Adelaide_Data.csv", gradients=False, rescale=True)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31]
[48]


In [55]:
dataset.n

71999

In [56]:
# data parameters
n   = dataset.n
print("n is: ", n)
dim = dataset.dim
print("dims is: ", dim)

# training params
num_inducing = 500
num_directions = 1
minibatch_size = 500
num_epochs = 100

# seed
torch.random.manual_seed(0)
# use tqdm or just have print statements
tqdm = False
# use data to initialize inducing stuff
inducing_data_initialization = False
# use natural gradients and/or CIQ
use_ngd = False
use_ciq = False
num_contour_quadrature=15
# learning rate
learning_rate_hypers = 0.01
learning_rate_ngd    = 0.1
gamma  = 10.0
#levels = np.array([20,150,300])
#def lr_sched(epoch):
#  a = np.sum(levels > epoch)
#  return (1./gamma)**a
lr_sched = None

n is:  71999
dims is:  32


In [57]:
# train-test split
n_train = int(0.8*dataset.n)
n_test  = n - n_train
train_dataset,test_dataset = torch.utils.data.random_split(dataset,[n_train,n_test])

In [58]:
#loaders
train_loader = DataLoader(train_dataset, batch_size=minibatch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=n_test, shuffle=False)

In [59]:
test_y = [item[1] for item in test_loader]
test_x = [item[0] for item in test_loader]

# D-Free Grad SVGP

In [60]:
# train
print("\n\n---DirectionalGradVGP---")
print(f"Start training with {n} trainig data of dim {dim}")
print(f"VI setups: {num_inducing} inducing points, {num_directions} inducing directions")
args={"verbose":True}
t1 = time.time()	
model,likelihood = train_gp(train_dataset,
                      num_inducing=num_inducing,
                      num_directions=num_directions,
                      minibatch_size = minibatch_size,
                      minibatch_dim = num_directions,
                      num_epochs =num_epochs, 
                      learning_rate_hypers=learning_rate_hypers,
                      learning_rate_ngd=learning_rate_ngd,
                      inducing_data_initialization=inducing_data_initialization,
                      use_ngd = use_ngd,
                      use_ciq = use_ciq,
                      lr_sched=lr_sched,
                      num_contour_quadrature=num_contour_quadrature,
                      tqdm=tqdm,**args
                      )
t2 = time.time()	

# save the model
# torch.save(model.state_dict(), "../data/test_dvi_basic.model")

# test
means, variances = eval_gp( test_dataset,model,likelihood,
                            num_directions=num_directions,
                            minibatch_size=n_test,
                            minibatch_dim=num_directions)
t3 = time.time()	




---DirectionalGradVGP---
Start training with 71999 trainig data of dim 32
VI setups: 500 inducing points, 1 inducing directions
All parameters to learn:
      variational_strategy.inducing_points
      torch.Size([500, 32])
      variational_strategy.inducing_directions
      torch.Size([500, 32])
      variational_strategy._variational_distribution.variational_mean
      torch.Size([1000])
      variational_strategy._variational_distribution.chol_variational_covar
      torch.Size([1000, 1000])
      mean_module.constant
      torch.Size([1])
      covar_module.raw_outputscale
      torch.Size([])
      covar_module.base_kernel.raw_lengthscale
      torch.Size([1, 1])
      noise_covar.raw_noise
      torch.Size([1])
Total number of parameters:  1033004.0
Epoch: 0; total_step: 0, loss: 2.447085651562016, nll: 1.4299344319627434
Epoch: 0; total_step: 50, loss: 1.665164807180518, nll: 1.092660237192947
Epoch: 0; total_step: 100, loss: 1.4598424966029546, nll: 0.9278071120597614
Epoch:

Epoch: 40; total_step: 4750, loss: 0.7718113643685642, nll: 0.1827085693669864
Epoch: 41; total_step: 4800, loss: 0.7412004183555241, nll: 0.20920533949989548
Epoch: 41; total_step: 4850, loss: 0.710377068544935, nll: 0.14791816608257133
Epoch: 42; total_step: 4900, loss: 0.725999128885452, nll: 0.12053632683493268
Epoch: 42; total_step: 4950, loss: 0.745344284064476, nll: 0.17746799877755348
Epoch: 43; total_step: 5000, loss: 0.7382342326666669, nll: 0.19454226527399987
Epoch: 43; total_step: 5050, loss: 0.7599104263214327, nll: 0.17698582769802335
Epoch: 43; total_step: 5100, loss: 0.8183011556887363, nll: 0.24311971122740905
Epoch: 44; total_step: 5150, loss: 0.7390665783390111, nll: 0.17470662044583718
Epoch: 44; total_step: 5200, loss: 0.8321060805859929, nll: 0.21708296747803582
Epoch: 45; total_step: 5250, loss: 0.7783315597528027, nll: 0.17241484969133533
Epoch: 45; total_step: 5300, loss: 0.7234777173213294, nll: 0.13644292184833984
Epoch: 46; total_step: 5350, loss: 0.7149461

Epoch: 85; total_step: 9900, loss: 0.7603975044922776, nll: 0.1725347898539301
Epoch: 85; total_step: 9950, loss: 0.7406921914833374, nll: 0.13591764733525902
Epoch: 86; total_step: 10000, loss: 0.7222007188963467, nll: 0.18182704334608354
Epoch: 86; total_step: 10050, loss: 0.7023998272367388, nll: 0.1341520859722018
Epoch: 87; total_step: 10100, loss: 0.7022649781877653, nll: 0.1350191093854381
Epoch: 87; total_step: 10150, loss: 0.7142825846793518, nll: 0.15670719260646412
Epoch: 87; total_step: 10200, loss: 0.7408725067098202, nll: 0.15187528192956343
Epoch: 88; total_step: 10250, loss: 0.670246325327836, nll: 0.12234005179391386
Epoch: 88; total_step: 10300, loss: 0.8085302295743488, nll: 0.23242287360187106
Epoch: 89; total_step: 10350, loss: 0.678572661833214, nll: 0.1325017095485234
Epoch: 89; total_step: 10400, loss: 0.6596862978411705, nll: 0.1387181399880714
Epoch: 90; total_step: 10450, loss: 0.7807529013051421, nll: 0.12989073932517853
Epoch: 90; total_step: 10500, loss: 0

In [61]:

# compute MSE
#test_y = test_y.cpu()
test_mse = MSE(test_y[0],means)
# compute mean negative predictive density
test_nll = -torch.distributions.Normal(means, variances.sqrt()).log_prob(test_y[0]).mean()
print(f"At {n_test} testing points, MSE: {test_mse:.4e}, nll: {test_nll:.4e}.")
print(f"Training time: {(t2-t1):.2f} sec, testing time: {(t3-t2):.2f} sec")

#plot=1
#if plot == 1:
#    from mpl_toolkits.mplot3d import axes3d
#    import matplotlib.pyplot as plt
#    fig = plt.figure(figsize=(12,6))
#    ax = fig.add_subplot(111, projection='3d')
#    ax.scatter(test_x[0][:,0],test_x[:,1],test_y, color='k')
#    ax.scatter(test_x[0][:,0],test_x[:,1],means, color='b')
#    plt.title("f(x,y) variational fit; actual curve is black, variational is blue")
#    plt.show()

At 14400 testing points, MSE: 8.9581e-02, nll: 2.0471e-01.
Training time: 22806.69 sec, testing time: 20.81 sec


In [62]:
# training params
#num_inducing = 50
#num_directions = 6
#minibatch_size = 200
#num_epochs = 100


# 2 directions
#At 104 testing points, MSE: 2.9133e+00, nll: 3.3945e+00. 
# 3 directions
#At 104 testing points, MSE: 2.9455e+00, nll: 3.3617e+00.
#Training time: 70.29 sec, testing time: 0.10 sec
# 4 directions
#At 104 testing points, MSE: 2.9810e+00, nll: 3.0743e+00.
#Training time: 57.68 sec, testing time: 0.08 sec
# 5 directions
#At 104 testing points, MSE: 2.9440e+00, nll: 3.6124e+00.
#Training time: 104.46 sec, testing time: 0.12 sec
# 6 directions
#At 104 testing points, MSE: 2.9795e+00, nll: 3.1092e+00.
#Training time: 127.73 sec, testing time: 0.10 sec
# 7 directions
#At 104 testing points, MSE: 2.9272e+00, nll: 3.6537e+00.
#Training time: 153.38 sec, testing time: 0.12 sec
# 8 directions
#At 104 testing points, MSE: 2.9503e+00, nll: 3.3300e+00.
#Training time: 173.86 sec, testing time: 0.15 sec
# 9 directions
# 10 directions

# Traditional SVGP

In [63]:
model_t,likelihood_t = traditional_vi.train_gp(train_dataset,dim,
                                                   num_inducing=num_inducing,
                                                   minibatch_size=minibatch_size,
                                                   num_epochs=num_epochs,
                                                   use_ngd=use_ngd, use_ciq=use_ciq,
                                                   learning_rate_hypers=learning_rate_hypers,
                                                   learning_rate_ngd=learning_rate_ngd,
                                                   lr_sched=lr_sched,
                                                   num_contour_quadrature=num_contour_quadrature,gamma=gamma, verbose=True)

All parameters to learn:
      variational_strategy.inducing_points
      torch.Size([500, 32])
      variational_strategy._variational_distribution.variational_mean
      torch.Size([500])
      variational_strategy._variational_distribution.chol_variational_covar
      torch.Size([500, 500])
      mean_module.constant
      torch.Size([1])
      covar_module.raw_outputscale
      torch.Size([])
      covar_module.base_kernel.raw_lengthscale
      torch.Size([1, 1])
      noise_covar.raw_noise
      torch.Size([1])
Total number of parameters:  266504.0
Using ELBO
Epoch: 0; total_step: 0, loss: 2.4303488237156334, nll: 1.429625159903971
Epoch: 0; total_step: 50, loss: 1.7147404010280134, nll: 1.1300444980506033
Epoch: 0; total_step: 100, loss: 1.5556107125336878, nll: 0.9989157119957579
Epoch: 1; total_step: 150, loss: 1.4231348405147806, nll: 0.8639497738756452
Epoch: 1; total_step: 200, loss: 1.3466182210865958, nll: 0.7648590500109362
Epoch: 2; total_step: 250, loss: 1.2476403276082

Epoch: 41; total_step: 4850, loss: 0.9223424861270223, nll: 0.25638188013211727
Epoch: 42; total_step: 4900, loss: 0.9419478549036264, nll: 0.26712374072664824
Epoch: 42; total_step: 4950, loss: 0.9773105069237997, nll: 0.29751672819740466
Epoch: 43; total_step: 5000, loss: 0.9798422697344326, nll: 0.29530082362359245
Epoch: 43; total_step: 5050, loss: 0.9183639196540692, nll: 0.24068976593795036
Epoch: 43; total_step: 5100, loss: 0.9274180324566742, nll: 0.2546063756536531
Epoch: 44; total_step: 5150, loss: 0.9030776230154012, nll: 0.23387604366077053
Epoch: 44; total_step: 5200, loss: 0.9369213028681934, nll: 0.26292644456533537
Epoch: 45; total_step: 5250, loss: 0.9215193430522371, nll: 0.246968391413592
Epoch: 45; total_step: 5300, loss: 0.8897328585829688, nll: 0.22046214612079512
Epoch: 46; total_step: 5350, loss: 0.9771219167613898, nll: 0.2963244062538112
Epoch: 46; total_step: 5400, loss: 0.9457091975289484, nll: 0.26742014796221136
Epoch: 46; total_step: 5450, loss: 0.9138579

Epoch: 86; total_step: 10050, loss: 0.949318373185752, nll: 0.26656497379845007
Epoch: 87; total_step: 10100, loss: 0.9508156170803036, nll: 0.2636665323339685
Epoch: 87; total_step: 10150, loss: 0.9161823652418905, nll: 0.23945324037974397
Epoch: 87; total_step: 10200, loss: 0.9776420967811593, nll: 0.2808393219897056
Epoch: 88; total_step: 10250, loss: 0.9488860061051805, nll: 0.26578054803700446
Epoch: 88; total_step: 10300, loss: 0.9167577537761356, nll: 0.24069635385202
Epoch: 89; total_step: 10350, loss: 0.9236188733873494, nll: 0.24834209503629032
Epoch: 89; total_step: 10400, loss: 0.9563813673459732, nll: 0.2580262920012555
Epoch: 90; total_step: 10450, loss: 0.9141946917726824, nll: 0.2346206732162872
Epoch: 90; total_step: 10500, loss: 0.9667445556422742, nll: 0.2777008644322143
Epoch: 90; total_step: 10550, loss: 0.9532132950934878, nll: 0.26454712536071456
Epoch: 91; total_step: 10600, loss: 0.9159104734295851, nll: 0.23425026204353316
Epoch: 91; total_step: 10650, loss: 0

In [64]:
means_t, variances_t = traditional_vi.eval_gp(test_dataset, model_t, likelihood_t, minibatch_size=n_test)

In [65]:
# compute MSE
#test_y = test_y.cpu()
test_mse = MSE(test_y[0],means_t)
# compute mean negative predictive density
test_nll = -torch.distributions.Normal(means_t, variances_t.sqrt()).log_prob(test_y[0]).mean()
print(f"At {n_test} testing points, MSE: {test_mse:.4e}, nll: {test_nll:.4e}.")
print(f"Training time: {(t2-t1):.2f} sec, testing time: {(t3-t2):.2f} sec")

At 14400 testing points, MSE: 9.7147e-02, nll: 2.6375e-01.
Training time: 22806.69 sec, testing time: 20.81 sec


In [66]:
# protein
# dfree and svgp
# At 9146 testing points, MSE: 5.9555e-01, nll: 1.1599e+00.
# At 9146 testing points, MSE: 6.2736e-01, nll: 1.1856e+00.

In [67]:
#forest fire

# At 104 testing points, MSE: 3.0218e+00, nll: 4.2752e+00.
#Training time: 837.60 sec, testing time: 0.31 sec

#At 104 testing points, MSE: 2.8974e+00, nll: 3.5117e+00.
#Training time: 837.60 sec, testing time: 0.31 sec

In [ ]:
#At 14400 testing points, MSE: 8.9581e-02, nll: 2.0471e-01.
#At 14400 testing points, MSE: 9.7147e-02, nll: 2.6375e-01.